<a href="https://colab.research.google.com/github/wangjb/cwb_open_data/blob/master/cl_30days_meteo_manned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def retrieve_data(datacode,auth):
  import urllib
  # query string to CWB open data service
  url='https://opendata.cwb.gov.tw/'
  sub_path='fileapi/v1/opendataapi/'
  # instant 10 min precip
  datalimit=''
  dataoffset=''
  dataformat='JSON'
  stationID=''
  datastatus=''

  # create query string
  url_path=url+sub_path+datacode
  query_fields=['Authorization','limit','offset','format','stationID','status']
  query_data=(auth,datalimit,dataoffset,dataformat,stationID,datastatus)
  query_string=dict(zip(query_fields,query_data))

  # request data
  import requests
  r = requests.get(url_path,params=query_string,stream=True)
  jsondata = r.json()

  return jsondata

In [2]:
datacode = 'C-B0024-001'
auth = None
jsondata = retrieve_data(datacode, auth)

In [3]:
import pandas as pd
data = jsondata['cwbdata']['resources']['resource']['data']['surfaceObs']['location']

In [4]:
data[0].keys()

dict_keys(['station', 'stationObsTimes', 'stationObsStatistics'])

In [5]:
# process station info
station = pd.DataFrame(list(map(
    lambda x : x['station'],
      data)))

In [6]:
# process stationObsTimes	
stationObsTime = pd.DataFrame.from_dict({ (i,w['dataTime']) : w['weatherElements']
            for i,v in enumerate(data)
            for j,w in enumerate(v['stationObsTimes']['stationObsTime'])
          },orient='Index')

In [7]:
# process stationObsStatistics	
from itertools import islice
stationObsStatistics = pd.DataFrame.from_dict({ (i,w['dataDate']) : dict(islice(w.items(), 1, None))
              for i,v in enumerate(data)
              for j,w in enumerate(v['stationObsStatistics']['temperature']['daily'])
            },orient='Index')

### Variable "station"

In [8]:
station

,stationID,stationName,stationNameEN,stationAttribute
0,466880,板橋,BANQIAO,局屬有人氣象站
1,466900,淡水,TAMSUI,局屬有人氣象站
2,466910,鞍部,ANBU,局屬有人氣象站
3,466920,臺北,TAIPEI,局屬有人氣象站
4,466930,竹子湖,ZHUZIHU,局屬有人氣象站
5,466940,基隆,KEELUNG,局屬有人氣象站
6,466950,彭佳嶼,PENGJIAYU,局屬有人氣象站
7,466990,花蓮,HUALIEN,局屬有人氣象站
8,467050,新屋,XINWU,局屬有人氣象站
9,467060,蘇澳,SU-AO,局屬有人氣象站


### Variable "stationObsTimes"

In [9]:
stationObsTime

stationPressure  ... sunshineDuration
0  2020-07-13T01:00:00+08:00          1006.4  ...             None
   2020-07-13T02:00:00+08:00          1006.1  ...             None
   2020-07-13T03:00:00+08:00          1006.2  ...             None
   2020-07-13T04:00:00+08:00          1006.3  ...             None
   2020-07-13T05:00:00+08:00          1006.4  ...             None
...                                      ...  ...              ...
29 2020-08-12T20:00:00+08:00          1003.7  ...             None
   2020-08-12T21:00:00+08:00          1004.6  ...             None
   2020-08-12T22:00:00+08:00          1004.8  ...             None
   2020-08-12T23:00:00+08:00          1004.6  ...             None
   2020-08-12T24:00:00+08:00          1004.3  ...             None

[22272 rows x 7 columns]

### Varaible "stationObsStatistics"

In [10]:
stationObsStatistics

maximum minimum  mean
0  2020-07-13    38.2    27.1  31.9
   2020-07-14    36.8    27.0  31.5
   2020-07-15    36.2    28.3  31.3
   2020-07-16    36.8    28.0  31.3
   2020-07-17    38.0    26.2  31.8
...               ...     ...   ...
29 2020-08-08    35.3    26.8  29.6
   2020-08-09    30.9    26.2  27.8
   2020-08-10    33.0    26.8  28.9
   2020-08-11    33.5    24.3  27.2
   2020-08-12    32.4    25.7  28.0

[928 rows x 3 columns]